<a href="https://colab.research.google.com/github/Five-Star-Station/Hansol_QA/blob/main/dacon_hansolqa_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keybert

In [ ]:
!pip install kiwipiepy

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
DATA_PATH = '/content/'
train_df = pd.read_csv(DATA_PATH + 'train.csv')
test_df = pd.read_csv(DATA_PATH + 'test.csv')
submission_df = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [6]:
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [7]:
train_df['category'].value_counts()

마감재       272
인테리어      123
시공        111
마감하자       60
건축구조       31
기타         27
타 마감하자     20
Name: category, dtype: int64

### 키워드 추출

In [8]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

In [12]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [18]:
text = train_df['답변_5'][0]

keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=3)
keywords

[('진동 에너지의', 0.4812), ('진동의 영향을', 0.4738), ('에너지의 영향을', 0.4428)]

#### 명사만 추출

In [19]:
kiwi = Kiwi()
kiwi.analyze(text)

[([Token(form='면', tag='NNG', start=0, len=1),
   Token(form='지', tag='VV', start=1, len=1),
   Token(form='ᆫ', tag='ETM', start=1, len=1),
   Token(form='장치', tag='NNG', start=2, len=2),
   Token(form='는', tag='JX', start=4, len=1),
   Token(form='건물', tag='NNG', start=6, len=2),
   Token(form='에', tag='JKB', start=8, len=1),
   Token(form='오', tag='VV', start=10, len=1),
   Token(form='는', tag='ETM', start=11, len=1),
   Token(form='지반', tag='NNG', start=13, len=2),
   Token(form='진동', tag='NNG', start=16, len=2),
   Token(form='의', tag='JKG', start=18, len=1),
   Token(form='영향', tag='NNG', start=20, len=2),
   Token(form='을', tag='JKO', start=22, len=1),
   Token(form='최대한', tag='NNG', start=24, len=3),
   Token(form='으로', tag='JKB', start=27, len=2),
   Token(form='흡수', tag='NNG', start=30, len=2),
   Token(form='하', tag='XSV', start=32, len=1),
   Token(form='어', tag='EC', start=33, len=1),
   Token(form='건물', tag='NNG', start=35, len=2),
   Token(form='에', tag='JKB', start=37, l

키워드 추출을 위해 tag가 N으로 시작하는 명사만 남기자

In [20]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'): # 명사와 알파벳
            results.append(token)
    return results

In [21]:
nouns = noun_extractor(text)
nouns

['장치',
 '건물',
 '지반',
 '진동',
 '영향',
 '최대한',
 '흡수',
 '건물',
 '전달',
 '진동',
 '장치',
 '지반',
 '진동',
 '에너지',
 '영향',
 '완화',
 '사용']

인테리어 관련 전문 용어가 형태소 분석기에 없어서 `면진`을 추출하지 못한다...

In [23]:
text = ' '.join(nouns)
text

'장치 건물 지반 진동 영향 최대한 흡수 건물 전달 진동 장치 지반 진동 에너지 영향 완화 사용'

In [24]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('에너지', 0.4803),
 ('장치', 0.47),
 ('건물', 0.4483),
 ('진동', 0.3663),
 ('사용', 0.3437),
 ('영향', 0.3074),
 ('전달', 0.2588),
 ('흡수', 0.2558),
 ('완화', 0.2086),
 ('최대한', 0.1943),
 ('지반', 0.1522)]